In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

# Read the Data

In [2]:
data = pd.read_csv("train.csv")
data.head()
data_copy = data.copy()
data_copy['Tarih'] = pd.to_datetime(data_copy['Tarih'])

In [3]:
future_df = pd.read_csv("future.csv").drop('Unnamed: 0' , axis = 1)
future_df.head()

,Tarih,Dağıtılan Enerji (MWh)
0,2022-08-01 00:00:00,NaN
1,2022-08-01 01:00:00,NaN
2,2022-08-01 02:00:00,NaN
3,2022-08-01 03:00:00,NaN
4,2022-08-01 04:00:00,NaN


In [4]:
med = pd.read_csv('med.csv')
med['Tarih'] = pd.to_datetime(med['Tarih'])
med[med['Tarih'].dt.month == 8]

,Tarih
18,2021-08-02
29,2021-08-03


# Describe the Data

In [5]:
data.shape

(40152, 2)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40152 entries, 0 to 40151
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Tarih                   40152 non-null  object 
 1   Dağıtılan Enerji (MWh)  40152 non-null  float64
dtypes: float64(1), object(1)
memory usage: 627.5+ KB


In [7]:
data['Tarih'] = pd.to_datetime(data['Tarih'])
future_df['Tarih'] = pd.to_datetime(future_df['Tarih'])
# data.set_index('Tarih', inplace=True)
# future_df.set_index('Tarih' , inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40152 entries, 0 to 40151
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Tarih                   40152 non-null  datetime64[ns]
 1   Dağıtılan Enerji (MWh)  40152 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 627.5 KB


In [8]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Dağıtılan Enerji (MWh),40152.0,1836.805287,426.066085,870.18328,1499.165048,1813.409221,2129.407998,3633.105297


# Plot the Data

# ANN Model

## Train | Test Split

In [9]:
import tensorflow as tf

# Seed ayarlamak
seed_value = 53
tf.keras.utils.set_random_seed(seed_value)

In [10]:
test_df = data.loc[data.Tarih > '2022-07-23 23:00:00'].copy()
train_df = data.loc[data.Tarih <= '2022-07-23 23:00:00'].copy()

val_df = train_df[-96:]
train_df = train_df[:-96]

In [11]:
print(f"X_train max : {train_df.max()}\n\n")
print(f"X_val max : {val_df.max()}")
print(f"X_test max : {test_df.max()}\n\n\n\n")

print(f"X_train min : {train_df.min()}\n\n")
print(f"X_val min : {val_df.min()}")
print(f"X_test min : {test_df.min()}")

X_train max : Tarih                     2022-07-19 23:00:00
Dağıtılan Enerji (MWh)            3633.105297
dtype: object


X_val max : Tarih                     2022-07-23 23:00:00
Dağıtılan Enerji (MWh)            2812.085678
dtype: object
X_test max : Tarih                     2022-07-31 23:00:00
Dağıtılan Enerji (MWh)            3079.546897
dtype: object




X_train min : Tarih                     2018-01-01 00:00:00
Dağıtılan Enerji (MWh)              870.18328
dtype: object


X_val min : Tarih                     2022-07-20 00:00:00
Dağıtılan Enerji (MWh)            1596.356554
dtype: object
X_test min : Tarih                     2022-07-24 00:00:00
Dağıtılan Enerji (MWh)            1594.214702
dtype: object


# Feature Creators

In [12]:
all_df = [train_df , val_df , test_df , future_df , data]

In [13]:
from eval_metrics import eval_metrics
from feat_engs import create_datetimes, create_holiday_weekend, create_electricOutage_timeofDay, create_businessDay_cumulativeholidays, create_outage_rolling_percentages, create_hourly_sin_cos, create_seasons

In [14]:
for df in all_df:
    create_datetimes(df)
    create_holiday_weekend(df)
    create_electricOutage_timeofDay(df,med)
    create_businessDay_cumulativeholidays(df)
    create_outage_rolling_percentages(df)
    create_hourly_sin_cos(df)
    create_seasons(df)
    df.drop('date' , axis=1 , inplace= True)
    df.set_index('Tarih' , inplace= True)

In [15]:
print(train_df.shape)
train_df.head(2)

(39864, 26)


,Dağıtılan Enerji (MWh),hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,holiday,...,rolling_outages_24h,exp_avg_outages_24h,hour_sin,hour_cos,is_spring,is_summer,is_autumn,is_winter,is_weekend,is_weekday
Tarih,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00,1593.944216,0,0,1,1,2018,1,1,1,1,...,0.0,0.0,0.000000,1.000000,0,0,0,0,0,-1
2018-01-01 01:00:00,1513.933887,1,0,1,1,2018,1,1,1,1,...,0.0,0.0,0.258819,0.965926,0,0,0,0,0,-1


In [16]:
print(val_df.shape)
val_df.head(2)

(96, 26)


,Dağıtılan Enerji (MWh),hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,holiday,...,rolling_outages_24h,exp_avg_outages_24h,hour_sin,hour_cos,is_spring,is_summer,is_autumn,is_winter,is_weekend,is_weekday
Tarih,,,,,,,,,,,,,,,,,,,,,
2022-07-20 00:00:00,1972.864041,0,2,3,7,2022,201,20,29,0,...,0.0,0.0,0.000000,1.000000,0,1,0,0,0,-1
2022-07-20 01:00:00,1833.400302,1,2,3,7,2022,201,20,29,0,...,0.0,0.0,0.258819,0.965926,0,1,0,0,0,-1


In [17]:
print(test_df.shape)
test_df.head(2)

(192, 26)


,Dağıtılan Enerji (MWh),hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,holiday,...,rolling_outages_24h,exp_avg_outages_24h,hour_sin,hour_cos,is_spring,is_summer,is_autumn,is_winter,is_weekend,is_weekday
Tarih,,,,,,,,,,,,,,,,,,,,,
2022-07-24 00:00:00,2055.091859,0,6,3,7,2022,205,24,29,0,...,0.0,0.0,0.000000,1.000000,0,1,0,0,0,-1
2022-07-24 01:00:00,1930.490383,1,6,3,7,2022,205,24,29,0,...,0.0,0.0,0.258819,0.965926,0,1,0,0,0,-1


In [18]:
print(future_df.shape)
future_df.head(2)

(744, 26)


,Dağıtılan Enerji (MWh),hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,holiday,...,rolling_outages_24h,exp_avg_outages_24h,hour_sin,hour_cos,is_spring,is_summer,is_autumn,is_winter,is_weekend,is_weekday
Tarih,,,,,,,,,,,,,,,,,,,,,
2022-08-01 00:00:00,NaN,0,0,3,8,2022,213,1,31,0,...,0.0,0.0,0.000000,1.000000,0,1,0,0,0,-1
2022-08-01 01:00:00,NaN,1,0,3,8,2022,213,1,31,0,...,0.0,0.0,0.258819,0.965926,0,1,0,0,0,-1


In [19]:
print(data.shape)
data.head(2)

(40152, 26)


,Dağıtılan Enerji (MWh),hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,holiday,...,rolling_outages_24h,exp_avg_outages_24h,hour_sin,hour_cos,is_spring,is_summer,is_autumn,is_winter,is_weekend,is_weekday
Tarih,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00,1593.944216,0,0,1,1,2018,1,1,1,1,...,0.0,0.0,0.000000,1.000000,0,0,0,0,0,-1
2018-01-01 01:00:00,1513.933887,1,0,1,1,2018,1,1,1,1,...,0.0,0.0,0.258819,0.965926,0,0,0,0,0,-1


In [20]:
future_df.isna().sum()

Dağıtılan Enerji (MWh)    744
hour                        0
dayofweek                   0
quarter                     0
month                       0
year                        0
dayofyear                   0
dayofmonth                  0
weekofyear                  0
holiday                     0
weekend                     0
electrical_outage           0
timeofday                   0
business_day                0
cumulative_holidays         0
outage_percentage           0
rolling_outages_24h         0
exp_avg_outages_24h         0
hour_sin                    0
hour_cos                    0
is_spring                   0
is_summer                   0
is_autumn                   0
is_winter                   0
is_weekend                  0
is_weekday                  0
dtype: int64

In [21]:
# cat_cols = ['hour', 'dayofweek', 'quarter', 'month', 'year', 'dayofyear', 'dayofmonth', 'weekofyear' , 'holiday' , 'weekend' , 'timeofday' , 'business_day' , 'is_spring' , 'is_summer' , 'is_autumn' , 'is_winter' , 'is_weekend' , 'is_weekday' ] 
# # rolling_outages_24h ? , is_spring , is_summer , is_autumn , is_winter , is_weekend , is_weekday
# train_df = pd.get_dummies(train_df, columns=cat_cols, drop_first=True)
# val_df = pd.get_dummies(val_df, columns=cat_cols, drop_first=True)
# test_df = pd.get_dummies(test_df, columns=cat_cols, drop_first=True)
# future_df = pd.get_dummies(future_df, columns=cat_cols, drop_first=True)

# Set X and Y

In [22]:
# X_train = train_df.drop('Dağıtılan Enerji (MWh)' , axis=1)
# y_train = train_df['Dağıtılan Enerji (MWh)']

# X_val = val_df.drop('Dağıtılan Enerji (MWh)' , axis=1)
# y_val = val_df['Dağıtılan Enerji (MWh)']

# X_test = test_df.drop('Dağıtılan Enerji (MWh)' , axis=1)
# y_test = test_df['Dağıtılan Enerji (MWh)']

# X_future = future_df.drop('Dağıtılan Enerji (MWh)' , axis=1)

In [23]:
# X_train.shape

In [24]:
# 1. Veri setini train, validation ve test olarak üç parçaya ayırın.
train_size = int(len(data) * 0.8)
val_size = int(len(data) * 0.1)
test_size = len(data) - train_size - val_size

train_data = data[:train_size]
val_data = data[train_size:train_size+val_size]
test_data = data[-test_size:]

In [25]:
data_arr = np.array(data)
train_data_arr = np.array(train_data)
val_data_arr = np.array(val_data)
test_data_arr = np.array(test_data)

In [26]:
# X_all = data.drop('Dağıtılan Enerji (MWh)' , axis = 1)
# y_all = data['Dağıtılan Enerji (MWh)']

# X_train = train_data.drop('Dağıtılan Enerji (MWh)' , axis = 1)
# y_train = train_data['Dağıtılan Enerji (MWh)']

# X_val = val_data.drop('Dağıtılan Enerji (MWh)' , axis = 1)
# y_val = val_data['Dağıtılan Enerji (MWh)']

# X_test = test_data.drop('Dağıtılan Enerji (MWh)' , axis = 1)
# y_test = test_data['Dağıtılan Enerji (MWh)']

In [27]:
# window_size = 24

# # Veri setini yeniden şekillendirme
# X_train = X_train.values.reshape((-1, window_size, X_train.shape[1]))
# X_val = X_val.values.reshape((-1, window_size, X_val.shape[1]))
# X_test = X_test.values.reshape((-1, window_size, X_test.shape[1]))

In [28]:
# X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
# X_val = X_val.values.reshape(X_val.shape[0], X_val.shape[1], 1)
# X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)
# X_future = X_future.values.reshape(X_future.shape[0], X_future.shape[1], 1)

In [30]:
data_arr.shape

(40152, 26)

## Scaling

In [31]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data_arr_scaled = scaler.fit_transform(data_arr.reshape(-1,1))
train_data_arr_scaled = scaler.transform(train_data_arr.reshape(-1,1))
val_data_arr_scaled = scaler.transform(val_data_arr.reshape((-1,1)))
test_data_arr_scaled = scaler.transform(test_data_arr.reshape(-1,1))

In [ ]:
data_arr_scaled = scaler.fit_transform(data_arr.reshape(-1,1)) # veriyi scale et
data_arr = scaler.inverse_transform(data_arr_scaled).flatten() # orijinal veriye geri dönüştür

In [33]:
data_arr_scaled.shape

(1043952, 1)

In [ ]:
# # 2. Veri setini ölçeklendirin.
# from sklearn.preprocessing import MinMaxScaler

# scaler_X = MinMaxScaler()
# X_all = scaler_X.fit_transform(X_all.values)
# X_train = scaler_X.transform(X_train.values)
# X_val = scaler_X.transform(X_val.values)
# X_test = scaler_X.transform(X_test.values)


# scaler_y = MinMaxScaler()
# y_all = scaler_y.fit_transform(y_all.values.reshape(-1,1))
# y_train = scaler_y.transform(y_train.values.reshape(-1,1))
# y_val = scaler_y.transform(y_val.values.reshape(-1,1))
# y_test = scaler_y.transform(y_test.values.reshape(-1,1))

In [ ]:
# data = np.array(np.concatenate([X_all , y_all] , axis = 1))
# train_data = np.array(np.concatenate([X_train , y_train] , axis = 1))
# val_data = np.array(np.concatenate([X_val , y_val] , axis = 1))
# test_data = np.array(np.concatenate([X_test , y_test] , axis = 1))

In [ ]:
data_arr.shape

In [ ]:
data_arr_scaled.shape

In [ ]:
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler(feature_range=(0,1))
# data_scaled = scaler.fit_transform(data)
# train_scaled = scaler.transform(train_df)
# test_scaled = scaler.transform(test_df)

# val_scaled = scaler.transform(val_df)

In [ ]:
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler(feature_range=(0,1))
# X_train = scaler.fit_transform(X_train)
# X_val = scaler.transform(X_val)
# X_test = scaler.transform(X_test)

# X_future = scaler.transform(X_future)

## Packager

In [ ]:
def create_time_series(data, seq_length):
    # Veri boyutunu al
    data_len = len(data)
    
    # Pencereler için dizi oluştur
    windows = []
    
    # Veriyi pencere boyutuna göre parçalayarak windows dizisine ekle
    for i in range(seq_length, data_len+1):
        windows.append(data[i-seq_length:i, :])
    
    # Diziyi numpy dizisine dönüştür
    windows = np.array(windows)
    
    return windows

In [ ]:
data_seq = create_time_series(data_arr_scaled, 24)
train_seq = create_time_series(train_data_arr_scaled, 24)
val_seq = create_time_series(val_data_arr_scaled, 24)
test_seq = create_time_series(test_data_arr_scaled, 24)

In [ ]:
data_seq.shape

In [ ]:
val_seq.shape

In [ ]:
data_seq

In [ ]:
# scaler = StandardScaler()
# data_seq_scaled = scaler.fit_transform(data_seq)

In [ ]:
# X ve y değerlerini oluşturun
# X_all = data_seq[:, : , 1:]
y = data_seq[:, : , 0]

# X_train = train_seq[:, :, 1:]
y_train = train_seq[:, : , 0]

# X_val = val_seq[:, :, 1:]
y_val = val_seq[:, : , 0]

# X_test = test_seq[:, :, 1:]
y_test = test_seq[:, : , 0]

In [ ]:
(data_arr_scaled).shape

In [ ]:
y.shape

In [ ]:
scaler.inverse_transform(data_arr_scaled)

In [ ]:
scaler.inverse_transform(y)

In [ ]:
# window_size = 24

# def create_dataset(X, y, window_size):
#     Xs, ys = [], []
#     for i in range(len(X) - window_size):
#         Xs.append(X[i:i+window_size])
#         ys.append(y[i+window_size])
#     return np.array(Xs), np.array(ys)

# X_all, y_all = create_dataset(data[:, :-1], data[:, -1], window_size)
# X_train, y_train = create_dataset(train_data[:, :-1], train_data[:, -1], window_size)
# X_val, y_val = create_dataset(val_data[:, :-1], val_data[:, -1], window_size)
# X_test, y_test = create_dataset(test_data[:, :-1], test_data[:, -1], window_size)

In [ ]:
# X_all.shape

In [ ]:
# def packager(data_scaled , train_scaled, val_scaled, test_scaled, window_size=24):
#     X_data , y_data , X_train, y_train, X_val, y_val, X_test, y_test = [], [], [], [], [], [], [], []
    
    
#     for i in range(window_size, len(data_scaled)):
#         X_data.append(data_scaled[i-window_size:i, 0])
#         y_data.append(data_scaled[i, 0])

#     for i in range(window_size, len(train_scaled)):
#         X_train.append(train_scaled[i-window_size:i, 0])
#         y_train.append(train_scaled[i, 0])

#     for i in range(window_size, len(val_scaled)):
#         X_val.append(val_scaled[i-window_size:i, 0])
#         y_val.append(val_scaled[i, 0])

#     for i in range(window_size, len(test_scaled)):
#         X_test.append(test_scaled[i-window_size:i, 0])
#         y_test.append(test_scaled[i, 0])

#     X_data, y_data = np.array(X_data), np.array(y_data)
#     X_train, y_train = np.array(X_train), np.array(y_train)
#     X_val, y_val = np.array(X_val), np.array(y_val)
#     X_test, y_test = np.array(X_test), np.array(y_test)

#     X_data = np.reshape(X_data, (X_data.shape[0], X_train.shape[1], 1))
#     X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
#     X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))
#     X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

#     X = np.concatenate((X_train, X_val, X_test), axis=0)
#     y = np.concatenate((y_train, y_val, y_test), axis=0)

#     return X_data , y_data , X_train, y_train, X_val, y_val, X_test, y_test, X, y

In [ ]:
# X_data , y_data , X_train, y_train, X_val, y_val, X_test, y_test, X, y = packager(data , train_df, val_df, test_df)

In [ ]:
# print(f"X_data Shape : {X_data.shape}")
# print(f"X_train Shape : {X_train.shape}")
# print(f"X_test Shape : {X_test.shape}")
# print(f"X_val Shape : {X_val.shape}")
# print(f"X Shape : {X.shape}")

# Modelling Time

## Manual Create the Model

In [ ]:
# X_train.shape

In [ ]:
# from keras.models import Sequential
# from keras.layers import LSTM, Dense, Dropout, BatchNormalization
# from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau , EarlyStopping

# window_size = 24
# batch_size = 16


# # Model oluşturma
# model = Sequential()
# model.add(LSTM(256, input_shape=(window_size, data_seq.shape[2]-1), return_sequences=True))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))
# model.add(LSTM(128, return_sequences=True))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))
# model.add(LSTM(64, return_sequences=False))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))
# model.add(Dense(1))

# model.compile(loss='mse', optimizer='adam')


# reduce_lr = ReduceLROnPlateau(factor=0.1, patience=4)
# early_stop = EarlyStopping(monitor='val_loss', patience=7)
# model_check = ModelCheckpoint(filepath= r'C:\Users\Emincan\Desktop\Gdz\gdz-competition\Checkpoints\Multi_Features\256-128-64(Batch-Dropout0.2)\model-{epoch:03d}-{val_loss:.5f}.h5', monitor="val_loss", verbose=1)

# # Modeli eğitme
# model.fit(X , y , epochs=10, batch_size=batch_size, validation_data=(X_val, y_val), callbacks=[reduce_lr, early_stop, model_check])


# # Modeli değerlendirme
# mse = model.evaluate(X_test, y_test)
# print('Test MSE: %.3f' % mse)


## Import the Model

In [ ]:
from tensorflow.keras.models import load_model

model = load_model(r"C:\Users\Emincan\Desktop\Gdz\gdz-competition\Checkpoints\Multi_Features\256-128-64(Batch-Dropout0.2)\\model-001-0.09063.h5")

# Predictions

In [ ]:
data.shape

In [ ]:
X_train.shape

In [ ]:
X_val.shape

In [ ]:
X_test.shape

In [ ]:
X_all.shape

In [ ]:
data_last_seq = X_all[-1:]
data_last_seq.shape

In [ ]:
data_last_seq.shape

In [ ]:
# Son pencerenin son durumunu tutmak için bir değişken oluşturun
last_window = data_last_seq.copy()

# 31 gün boyunca tahminler yapmak için bir boş tahmin dizisi oluşturun
predictions = []

for i in range(31*24):
    # Tahmin edilen değerleri ölçeklendirmek için son pencereyi yeniden şekillendirin
    last_window_reshaped = last_window.reshape(1, 24, 25)

    # Son pencereyi kullanarak bir tahmin yapın
    prediction = model.predict(last_window_reshaped , verbose = 0)[0][0]

    # Tahmini tahmin listesine ekleyin
    predictions.append(prediction)

    # Tahmin edilen değeri son pencerenin sonuna ekleyin
    last_window = np.roll(last_window, -1, axis=1)
    last_window[0][-1] = prediction

predictions = np.array(predictions).reshape(-1, 1)
predictions = scaler_y.inverse_transform(predictions)

## Visualize Predictions

In [ ]:
future_df['Dağıtılan Enerji (MWh)'] = predictions
future_df

In [ ]:
target_data = data_copy[data_copy['Tarih'].dt.month == 8]

In [ ]:
target_data_2018 = target_data[target_data.Tarih.dt.year == 2018]
target_data_2019 = target_data[target_data.Tarih.dt.year == 2019]
target_data_2020 = target_data[target_data.Tarih.dt.year == 2020]
target_data_2021 = target_data[target_data.Tarih.dt.year == 2021]

In [ ]:
target_data.mean()

In [ ]:
future_df['Dağıtılan Enerji (MWh)'].mean()

In [ ]:
data = data_copy.set_index('Tarih', drop=True)
future_data = future_df[['Dağıtılan Enerji (MWh)']]
target_data = target_data.set_index('Tarih' , drop=True)

In [ ]:
from eval_metrics import preds_plot

plt.figure(figsize=(16,12))
preds_plot(data , future_df['Dağıtılan Enerji (MWh)'], target_data )